## Kiva Loan Funding - PreProcessing

**PURPOSE**: Predict which microfinance loans will be funded 
**AUTHOR** : Maureen Wiebe

**DATA SOURCES**:<br> 
- Kiva Developer Tools: https://www.kiva.org/build/data-snapshots
    
**REV DATE**: 5-4-2021

In [9]:
import pandas as pd 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle 
plt.style.use('seaborn')
pd.set_option('max_columns', None)

In [10]:
loans_rand = pd.read_pickle('C:/Users/mwalz2/Documents/Python/Springboard/Kiva_Capstone_Project/data/interim/loans_rand_clean.pkl')

### Missing Values 
Most models cannot handle missing values. The Partner ID column contains missing values for "direct" loans.  Loans that are directly given to the borrower, rather than going through an intermediary. These values will be replaced with 0 as a placeholder because this value will be treated as categorical and encoded during the next step.

In [11]:
loans_rand.loc[loans_rand.PARTNER_ID.isnull(),'PARTNER_ID'] = 0 

### Categorical Features 
The following freatures were selected to be one-hot encoded in addition to the features that were selected prior: 
1. PARTNER_ID
2. DISTRIBUTION_MODEL
3. REPAYMENT_INTERVAL
4. COUNTRY_CODE
5. CURRENCY
7. SECTOR_NAME
8. ORIGINAL_LANGUAGE

Several other categorical features (ex. [Loan]Description, Borrower Names, etc.) were not selected to be included as part of the dataset for modeling because without further processing they would not add additional knowledge to the model. 

In [12]:
model_loans = pd.get_dummies(loans_rand, columns =['PARTNER_ID','DISTRIBUTION_MODEL','REPAYMENT_INTERVAL','COUNTRY_CODE','CURRENCY','SECTOR_NAME','ORIGINAL_LANGUAGE'], 
                            prefix=['Partner','Dist_Model','Repayment', 'Country', 'Currency', 'Sector','Post_Language'],drop_first = True)

### Modeling Dataset
Additional numeric columns were eliminated because they provided information to the model that would not be known when determining if the loan was ultimately funded.  For example the total number of lenders or the amount of time to raise the loan funds. Other columns with duplicative information were excluded for the modeling. 

The feature set as well as the target were designated as X, y respectively.  


In [13]:
X = model_loans.drop(['LOAN_ID','FUNDED_AMOUNT','FUNDED_PERCENT','CURRENCY_EXCHANGE_COVERAGE_RATE','RAISED_TIME','RAISED_HOURS','NUM_LENDERS_TOTAL','Status_funded','AVG_LENDER_AMT','POSTED_TIME','PLANNED_EXPIRATION_TIME','DISBURSE_TIME'],axis =1).select_dtypes(exclude ='object')
y = model_loans['Status_funded']

In [14]:
X.describe()

,LOAN_AMOUNT,CURRENCY_EXCHANGE_RATE_CALC,MONTH,LENDER_TERM,NUM_JOURNAL_ENTRIES,NUM_BULK_ENTRIES,NUM_BORROWERS_TOTAL,FEMALE_ONLY_LOAN,Tag_Low-profit FP,Tag_Woman Owned Biz,Tag_Parent,Tag_Repeat Borrower,Tag_Schooling,Tag_Biz Durable Asset,Tag_Vegan,Tag_user_favorite,Tag_Eco-friendly,Tag_Technology,Tag_Health and Sanitation,Tag_Animals,Tag_volunteer_pick,Tag_Hidden Gem,Tag_Interesting Photo,Tag_Single,Tag_Sustainable Ag,Tag_volunteer_like,Tag_First Loan,Tag_Single Parent,Tag_Elderly,Tag_Fabrics,Tag_Supporting Family,Tag_Female Education,Tag_Repair Renew Replace,Tag_Job Creator,Tag_Widowed,Tag_Inspiring Story,Tag_Trees,Tag_Unique,Tag_Refugee,Tag_Post-disbursed,Tag_Powerful Story,Tag_Orphan,Tag_US immigrant,Tag_Tourism,Tag_Married,Tag_Team Guys Holding Fish,Activity_Air Conditioning,Activity_Animal Sales,Activity_Aquaculture,Activity_Arts,Activity_Auto Repair,Activity_Bakery,Activity_Balut-Making,Activity_Barber Shop,Activity_Beauty Salon,Activity_Beekeeping,Activity_Beverages,Activity_Bicycle Repair,Activity_Bicycle Sales,Activity_Blacksmith,Activity_Bookbinding,Activity_Bookstore,Activity_Bricks,Activity_Butcher Shop,Activity_Cafe,Activity_Call Center,Activity_Carpentry,Activity_Catering,Activity_Cattle,Activity_Celebrations,Activity_Cement,Activity_Cereals,Activity_Charcoal Sales,Activity_Cheese Making,Activity_Child Care,Activity_Cleaning Services,Activity_Cloth & Dressmaking Supplies,Activity_Clothing,Activity_Clothing Sales,Activity_Cobbler,Activity_Communications,Activity_Computer,Activity_Computers,Activity_Construction,Activity_Construction Supplies,Activity_Consumer Goods,Activity_Cosmetics Sales,Activity_Crafts,Activity_Dairy,Activity_Decorations Sales,Activity_Dental,Activity_Education provider,Activity_Electrical Goods,Activity_Electrician,Activity_Electronics Repair,Activity_Electronics Sales,Activity_Embroidery,Activity_Energy,Activity_Entertainment,Activity_Event Planning,Activity_Farm Supplies,Activity_Farming,Activity_Film,Activity_Fish Selling,Activity_Fishing,Activity_Florist,Activity_Flowers,Activity_Food,Activity_Food Market,Activity_Food Production/Sales,Activity_Food Stall,Activity_Fruits & Vegetables,Activity_Fuel/Firewood,Activity_Funerals,Activity_Furniture Making,Activity_Games,Activity_General Store,Activity_Goods Distribution,Activity_Grocery Store,Activity_Hardware,Activity_Health,Activity_Higher education costs,Activity_Home Appliances,Activity_Home Energy,Activity_Home Products Sales,Activity_Hotel,Activity_Internet Cafe,Activity_Jewelry,Activity_Knitting,Activity_Land Rental,Activity_Landscaping / Gardening,Activity_Laundry,Activity_Liquor Store / Off-License,Activity_Livestock,Activity_Machine Shop,Activity_Machinery Rental,Activity_Manufacturing,Activity_Medical Clinic,Activity_Metal Shop,Activity_Milk Sales,Activity_Mobile Phones,Activity_Mobile Transactions,Activity_Motorcycle Repair,Activity_Motorcycle Transport,Activity_Movie Tapes & DVDs,Activity_Music Discs & Tapes,Activity_Musical Instruments,Activity_Musical Performance,Activity_Natural Medicines,Activity_Office Supplies,Activity_Paper Sales,Activity_Party Supplies,Activity_Patchwork,Activity_Perfumes,Activity_Personal Care Products,Activity_Personal Expenses,Activity_Personal Housing Expenses,Activity_Personal Medical Expenses,Activity_Personal Products Sales,Activity_Pharmacy,Activity_Phone Accessories,Activity_Phone Repair,Activity_Phone Use Sales,Activity_Photography,Activity_Pigs,Activity_Plastics Sales,Activity_Poultry,Activity_Primary/secondary school costs,Activity_Printing,Activity_Property,Activity_Pub,Activity_Quarrying,Activity_Recycled Materials,Activity_Recycling,Activity_Religious Articles,Activity_Renewable Energy Products,Activity_Restaurant,Activity_Retail,Activity_Rickshaw,Activity_Secretarial Services,Activity_Services,Activity_Sewing,Activity_Shoe Sales,Activity_Souvenir Sales,Activity_Spare Parts,Activity_Sporting Good Sales,Activity_Tailoring,Activity_Taxi,Activity_Technology,Activity_Textiles,Activity_Timber Sales

### Splitting the Dataset: Train & Test
To ensure that we can reliably test how good the model is at predicting outcomes, the dataset will be split into a train set (for training the model) and a test set to calculate performance metrics on.  A 70/30 split will be used.  

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

### Data Scaling 
For some of the models that I will be testing having a scaled datasets allows for the models to provide better predictions.  When the features aren't scaled the model can place greater importance on features that have a larger scale.  For this dataset I will be using the Min Max scaler to normalize the data since several of the freatures do not follow a normal distribution.   

In [16]:
#fit the scaler to the train dataset 
norm = MinMaxScaler().fit(X_train)

#transform the train dataset 
X_train_norm = norm.transform(X_train)

#transform the test dataset based on the scaler fitted to the train dataset to prevent any data leakage
X_test_norm = norm.transform(X_test)

### Export Variables

In [19]:
with open('trainingdatarand.pickle', 'wb') as f: 
    pickle.dump([X, y, X_train, X_test, y_train, y_test,X_train_norm,X_test_norm, norm], f)

In [18]:
model_loans.to_pickle('C:/Users/mwalz2/Documents/Python/Springboard/Kiva_Capstone_Project/data/interim/model_loans_rand.pkl')